In [ ]:
import argparse
import torch
import json
import numpy as np
from opts import *
from model import BinaryRNN
from prettytable import PrettyTable


def parse_option():
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    # Dataset
    parser.add_argument("--dataset", default="ISCXVPN2016",
                        choices=["ISCXVPN2016", "BOTIOT", "CICIOT2022", "PeerRush"])
    
    # Model options
    model_opts(parser)
    # Training options
    training_opts(parser)
    
    args = parser.parse_args()
    args.model_path = './save/{}/brnn_len{}_ipd{}_ev{}_hidden{}_{}/brnn-best'.format(
        args.dataset,
        args.len_embedding_bits, args.ipd_embedding_bits, args.embedding_vector_bits, args.rnn_hidden_bits,
        str(args.loss_factor) + '_' + str(args.focal_loss_gamma) + '_' + args.loss_type + '_' + str(args.learning_rate))
    with open('../dataset/{}/json/statistics.json'.format(args.dataset)) as fp:
        statistics = json.load(fp)
        args.labels_num = statistics['label_num']

    return args


def load_model(model, model_path):
    if hasattr(model, "module"):
        model.module.load_state_dict(torch.load(model_path, map_location="cpu"), strict=False)
    else:
        model.load_state_dict(torch.load(model_path, map_location="cpu"), strict=False)
    return model


def print_table(table, field_names=None, title=None):
    pretty_table = PrettyTable()
    if field_names is not None:
        pretty_table.field_names = field_names
    if title is not None:
        pretty_table.title = title
    pretty_table.add_rows(table)
    print(pretty_table)


def save_json(obj, filename):
    with open(filename, 'w') as f:
        json.dump(obj, f)


def load_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)


args = parse_option()
# build model
model = BinaryRNN(args)
load_model(model, args.model_path)
print(model)

In [ ]:
embedding_len_json = {
    'title': 'Embedding Layer of Packet Length',
    'field_names': ['In: Packet Length (0-1500)', 'Out: Length Embedding ({} bit)'.format(args.len_embedding_bits)],
    'table': []
}
for length in range(args.len_vocab):
    input = torch.LongTensor([length])
    len_emb = model.len_embedding(input)
    len_emb_bin = model.activation_quantizer(len_emb)
    output = ['1' if i == 1 else '0' for i in len_emb_bin[0]]
    embedding_len_json['table'].append([str(length), ''.join(output)])
print_table(embedding_len_json['table'], embedding_len_json['field_names'], embedding_len_json['title'])
save_json(embedding_len_json, '../p4/parameter/{}/embedding_len.json'.format(args.dataset))

In [ ]:
embedding_ipd_json = {
    'title': 'Embedding Layer of Inter Packet Delay',
    'field_names': ['In: Inter Packet Delay (0-2560)', 'Out: IPD Embedding ({} bit)'.format(args.ipd_embedding_bits)],
    'table': []
}
for ipd in range(args.ipd_vocab):
    input = torch.LongTensor([ipd])
    ipd_emb = model.ipd_embedding(input)
    ipd_emb_bin = model.activation_quantizer(ipd_emb)
    output = ['1' if i == 1 else '0' for i in ipd_emb_bin[0]]
    embedding_ipd_json['table'].append([str(ipd), ''.join(output)])
print_table(embedding_ipd_json['table'], embedding_ipd_json['field_names'], embedding_ipd_json['title'])
save_json(embedding_ipd_json, '../p4/parameter/{}/embedding_ipd.json'.format(args.dataset))

In [ ]:
fc_json = {
    'title': 'Fully Connected Layer of Embeddings',
    'field_names': ['In: IDP Embedding ({} bit)'.format(args.ipd_embedding_bits), 
                    'In: Length Embedding ({} bit)'.format(args.len_embedding_bits), 
                    'Out: Embedding Vector ({} bit)'.format(args.embedding_vector_bits)],
    'table': []
}
for ipd in range(2 ** args.ipd_embedding_bits):
    ipd_bin = '{0:b}'.format(ipd).zfill(args.ipd_embedding_bits)
    for length in range(2 ** args.len_embedding_bits):
        length_bin = '{0:b}'.format(length).zfill(args.len_embedding_bits)
        input = torch.Tensor([-1 if i == '0' else 1 for i in length_bin] + [-1 if i == '0' else 1 for i in ipd_bin])
        ev = model.fc(input)
        ev_bin = model.activation_quantizer(ev)
        output = ['1' if i == 1 else '0' for i in ev_bin]
        fc_json['table'].append([ipd_bin, length_bin, ''.join(output)])
print_table(fc_json['table'], fc_json['field_names'], fc_json['title'])
save_json(fc_json, '../p4/parameter/{}/fc.json'.format(args.dataset))

In [ ]:
init_gru_json = {
    'title': 'init GRU Layer',
    'field_names': ['In: Embedding Vector ({} bit)'.format(args.embedding_vector_bits),  
    'Out: Hidden States ({} bit)'.format(args.rnn_hidden_bits)],
    'table': []
}

for ev in range(2 ** args.embedding_vector_bits):
    ev_bin = '{0:b}'.format(ev).zfill(args.embedding_vector_bits)
    hidden_bin = '{0:b}'.format(0).zfill(args.rnn_hidden_bits)
    input_ev = torch.Tensor([-1 if i == '0' else 1 for i in ev_bin])
    input_hidden = torch.Tensor([0 if i == '0' else 1 for i in hidden_bin])
    current_hidden = model.binary_gru.cell(input_ev, input_hidden)
    current_hidden_bin = model.activation_quantizer(current_hidden)
    output = ['1' if i == 1 else '0' for i in current_hidden_bin]

    output = ['0'] * (9 - len(output)) + output
    
    init_gru_json['table'].append([ev_bin, ''.join(output)])
print_table(init_gru_json['table'], init_gru_json['field_names'], init_gru_json['title'])
save_json(init_gru_json, '../p4/parameter/{}/init_gru.json'.format(args.dataset))

In [ ]:
gru_json = {
    'title': 'GRU Layer',
    'field_names': ['In: Embedding Vector ({} bit)'.format(args.embedding_vector_bits), 
    'In: Last Hidden States ({} bit)'.format(args.rnn_hidden_bits), 
    'Out: Current Hidden States ({} bit)'.format(args.rnn_hidden_bits)],
    'table': []
}
for ev in range(2 ** args.embedding_vector_bits):
    ev_bin = '{0:b}'.format(ev).zfill(args.embedding_vector_bits)
    for hidden in range(2 ** args.rnn_hidden_bits):
        hidden_bin = '{0:b}'.format(hidden).zfill(args.rnn_hidden_bits)
        input_ev = torch.Tensor([-1 if i == '0' else 1 for i in ev_bin])
        input_hidden = torch.Tensor([-1 if i == '0' else 1 for i in hidden_bin])
        current_hidden = model.binary_gru.cell(input_ev, input_hidden)
        current_hidden_bin = model.activation_quantizer(current_hidden)
        output = ['1' if i == 1 else '0' for i in current_hidden_bin]

        output = ['0'] * (9 - len(output)) + output
        
        gru_json['table'].append([ev_bin, hidden_bin.zfill(9), ''.join(output)])
print_table(gru_json['table'], gru_json['field_names'], gru_json['title'])
save_json(gru_json, '../p4/parameter/{}/gru.json'.format(args.dataset))

In [ ]:
output_json = {
    'title': 'Output Layer',
    'field_names': ['In: Final Hidden States ({} bit)'.format(args.rnn_hidden_bits), 
                    'Out: Predicted Class', 
                    'Out: Quantified Probability (0-15)',
                    'Out: Probability Vector ({} class)'.format(args.labels_num)],
    'table': []
}
for final_hidden in range(2 ** args.rnn_hidden_bits):
    hidden_bin = '{0:b}'.format(final_hidden).zfill(args.rnn_hidden_bits)
    input = torch.Tensor([-1 if i == '0' else 1 for i in hidden_bin])
    scores = model.out_layer(input)
    PRs = torch.nn.functional.softmax(scores)
    C = int(torch.argmax(PRs))
    PR = int(float(PRs[C]) // (1 / 16))
    output_json['table'].append([hidden_bin.zfill(9), C, PR, [int(float(PRs[i]) // (1 / 16)) for i in range(len(PRs))]])
print_table(output_json['table'], output_json['field_names'], output_json['title'])
save_json(output_json, '../p4/parameter/{}/output.json'.format(args.dataset))